<a href="https://colab.research.google.com/github/dohyun1411/Quora-Insincere-Questions-Classification/blob/preprocessing1/qiqc_sub1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import gc
import random
import os

from tqdm.auto import tqdm
tqdm.pandas()

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import f1_score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from keras.layers import Dense, Input, Embedding, Dropout, Activation, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.optimizers import Adam
from keras.models import Model
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers

from keras.layers import *
from keras.models import *
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.initializers import *
from keras.optimizers import *
import keras.backend as K
from keras.callbacks import *
import tensorflow as tf

Config

In [ ]:
n_splits = 5
input_path = '../input/quora-insincere-questions-classification'
max_features = 120000 # None
embed_size = 300
maxlen = 70 # None
batch_size = 512
max_epoch = 10

Load train and test data

In [ ]:
train_path = os.path.join(input_path, 'train.csv')
test_path = os.path.join(input_path, 'test.csv')

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

Tokenize

In [ ]:
puncts = '´‘’“”…!#$%&()*+,-./:;<=>?@[\]^_`{|}~"' + "'"

In [ ]:
to_exclude = ''
to_tokenize = puncts

tokenizer = Tokenizer(num_words=max_features, filters=to_exclude, lower=False)

In [ ]:
train_text = train_df["question_text"].fillna("_na_")
test_text = test_df["question_text"].fillna("_na_")
train_y = train_df['target'].values

In [ ]:
train_text = train_text.progress_apply(lambda x: re.sub(r'(['+to_tokenize+'])', r' \1', x)).values
test_text = test_text.progress_apply(lambda x: re.sub(r'(['+to_tokenize+'])', r' \1', x)).values

In [ ]:
text = list(train_text) + list(test_text)

In [ ]:
tokenizer.fit_on_texts(text)
word_index = tokenizer.word_index

In [ ]:
train_X = tokenizer.texts_to_sequences(train_text)
test_X = tokenizer.texts_to_sequences(test_text)

train_X = pad_sequences(train_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)

In [ ]:
max_features = len(word_index) if max_features is None else max_features
# maxlen = max({len(seq) for seq in train_X} | {len(seq) for seq in val_X} | {len(seq) for seq in test_X}) if maxlen is None else maxlen

Load embedding

In [ ]:
import zipfile
from gensim.models import KeyedVectors

embeddings_path = os.path.join(input_path, 'embeddings.zip')

glove = 'glove.840B.300d/glove.840B.300d.txt'
wiki = 'wiki-news-300d-1M/wiki-news-300d-1M.vec'
google = 'GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin'
para = 'paragram_300_sl999/paragram_300_sl999.txt'


def load_embedding(embedding_name):

    def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')

    with zipfile.ZipFile(embeddings_path) as embeddings_zip:

        if embedding_name == google:
            return KeyedVectors.load_word2vec_format(embeddings_zip.open(google), binary=True)

        else:
            embedding = []
            for o in embeddings_zip.open(embedding_name):
                try:
                    if len(o.decode('utf-8')) > 100:
                        embedding.append(get_coefs(*o.decode('utf-8').split(" ")))
                except:
                    pass
        
        return dict(embedding)

In [ ]:
import nltk
# nltk.download('wordnet')

from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer
s = PorterStemmer()
l = LancasterStemmer()
n = WordNetLemmatizer()

def is_in_emb(word, i, embedding, emb_matrix):

    if word in embedding:
        emb_matrix[i] = embedding[word]
        return True
    
    tmp = word.lower()
    if tmp in embedding:
        emb_matrix[i] = embedding[tmp]
        return True
    
    tmp = word.upper()
    if tmp in embedding:
        emb_matrix[i] = embedding[tmp]
        return True

    tmp = word.capitalize()
    if tmp in embedding:
        emb_matrix[i] = embedding[tmp]
        return True

    tmp = s.stem(word)
    if tmp in embedding:
        emb_matrix[i] = embedding[tmp]
        return True
    
    tmp = l.stem(word)
    if tmp in embedding:
        emb_matrix[i] = embedding[tmp]
        return True
    
    tmp = n.lemmatize(word)
    if tmp in embedding:
        emb_matrix[i] = embedding[tmp]
        return True
    
    return False

In [ ]:
# puncts = '´‘’“”…!#$%&()*+,-./:;<=>?@[\]^_`{|}~"' + "'"

In [ ]:
def load_emb_matrix(embedding_name):

    embedding = load_embedding(embedding_name)
    if embedding_name == glove: emb_mean, emb_std = -0.005838499, 0.48782197
    if embedding_name == wiki: emb_mean, emb_std = -0.0033469985, 0.109855495
    if embedding_name == google: emb_mean, emb_std = -0.003527845, 0.13315111
    if embedding_name == para: emb_mean, emb_std = -0.0053248387, 0.49346521
    emb_matrix = np.random.normal(emb_mean, emb_std, (max_features, embed_size))
    for word, i in tqdm(word_index.items()):
        i -= 1
        assert i >= 0

        if i >= max_features: continue

        if is_in_emb(word, i, embedding, emb_matrix): continue

        if embedding_name == google:

            tmp = re.sub('[0-9]{5,}', '#####', word)
            tmp = re.sub('[0-9]{4}', '####', tmp)
            tmp = re.sub('[0-9]{3}', '###', tmp)
            tmp = re.sub('[0-9]{2}', '##', tmp)
            if is_in_emb(tmp, i, google_embedding, google_emb_matrix): continue

            for punct in puncts:
                tmp = tmp.replace(punct, '')
            if is_in_emb(tmp, i, google_embedding, google_emb_matrix): continue

        else:
        
            tmp = word
            for punct in puncts:
                tmp = tmp.replace(punct, '')
            if is_in_emb(tmp, i, embedding, emb_matrix): continue

            for num in '0123456789':
                tmp = tmp.replace(num, '')
            if is_in_emb(tmp, i, embedding, emb_matrix): continue
    

    del embedding
    gc.collect()
    return emb_matrix

In [ ]:
%%time
glove_emb_matrix = load_emb_matrix(glove)

In [ ]:
%%time
para_emb_matrix = load_emb_matrix(para)

In [ ]:
emb_matrix = np.mean((0.7 * glove_emb_matrix, 0.3 * para_emb_matrix), axis=0)

Train and Test

In [ ]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.embed = Embedding(max_features, embed_size, weights = [emb_matrix], trainable = False)
        self.rnn = Sequential([
            Bidirectional(LSTM(128, return_sequences = True)),
            Bidirectional(LSTM(128, return_sequences = True)),
            Conv1D(128, 3, activation = "relu"),
            GlobalMaxPool1D(),
            Dense(64, activation = "relu"),
            Dropout(0.5),
            Dense(32, activation = "relu"),
            Dropout(0.2),
            Dense(1, activation = "sigmoid")
        ])

    def call(self, inputs):
        x = self.embed(inputs)
        x = self.rnn(x)
        return x

In [ ]:
splits = list(StratifiedKFold(n_splits=n_splits, shuffle=True).split(train_X, train_y))
for i, (train_idx, val_idx) in enumerate(splits):
    print(f'Fold {i + 1}')

    model = MyModel()
    model.compile(loss = 'binary_crossentropy', optimizer = 'Adam', metrics = ['binary_accuracy'])

    train_X_fold = train_X[train_idx.astype(int)]
    train_y_fold = train_y[train_idx.astype(int)]
    val_X_fold = train_X[val_idx.astype(int)]
    val_y_fold = train_y[val_idx.astype(int)]

    model.fit(train_X_fold, train_y_fold, batch_size=batch_size, epochs=max_epochs, validation_data=(val_X_fold, val_y_fold))
    
    if i == 0:
        predicted = model.predict(test_X, batch_size=batch_size)
    else:
        predicted = predicted + model.predict(test_X, batch_size=batch_size)

predicted = predicted / len(splits)

In [ ]:
res = [0 if x < 0.4 else 1 for x in predicted]
submission = pd.DataFrame({'qid': test_df['qid'][:len(res)], 'prediction': res}, columns=['qid', 'prediction'])

In [ ]:
submission.to_csv('./submission.csv', index=False)